# Ejercicio, Regresión Lineal Simple.

Fuente: IBM Maraton 2020.

Explicación:

El reto de Digital House tiene que ver con la misión y visión de la empresa; la cual busca transformar la vida de las personas, desarrollando competencias digitales, para que generen un impacto positivo en la sociedad.

En ese sentido, Digital House busca a través de este desafío medir la empleabilidad de los cursos que disponibiliza en su plataforma. Es decir, qué tan factible es que un alumno o egresado de Digital House acceda al mercado laboral o consiga un trabajo en su área, luego de finalizar uno o varios de los cursos ofrecidos en la plataforma.

Entender qué características o variables hacen que una persona sea más o menos empleable es fundamental para la empresa y para generar ese impacto positivo que busca aportar a la sociedad.

Ejercicio modificado para clase de I.A. UACh - 2023, Luis Vidal (lvidal@uach.c)


# Paso 1. Carga de Librerias

In [ ]:
!pip install scikit-learn --upgrade
!pip install scipy --upgrade

     |████████████████████████████████| 23.2 MB 4.6 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1
     |████████████████████████████████| 38.2 MB 24 kB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


# Paso 2. Datos

Descarga desde GITHUB

In [ ]:
import pandas as pd
!wget --no-check-certificate --content-disposition https://raw.githubusercontent.com/vanderlei-test/654986294958/master/train_dataset_digitalhouse.csv

--2021-11-07 19:58:47--  https://raw.githubusercontent.com/vanderlei-test/654986294958/master/train_dataset_digitalhouse.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 654532 (639K) [text/plain]
Saving to: ‘train_dataset_digitalhouse.csv’

train_dataset_digit 100%[===================>] 639.19K  --.-KB/s    in 0.03s   

2021-11-07 19:58:47 (18.6 MB/s) - ‘train_dataset_digitalhouse.csv’ saved [654532/654532]



Carga de Datos en el Sistema

In [ ]:
df_training_dataset = pd.read_csv(r'train_dataset_digitalhouse.csv')
df_training_dataset.tail()

,Unnamed: 0,EDAD,GENERO,RESIDENCIA,NV_ESTUDIO,ESTUDIO_PREV,TRACK_DH,AVG_DH,MINUTES_DH,EXPERIENCIA,DIAS_EMP
8990,9995,29.0,MASCULINO,ARGENTINA,TERTIARY,DEVELOPMENT,NaN,4.0,4701.6,9.1,86.0
8991,9996,34.0,NaN,ARGENTINA,UNIVERSITARY,ENGINEERING,PROGRAMACION,3.4,4646.2,16.8,95.0
8992,9997,28.0,FEMENINO,ARGENTINA,POST_GRADUATE,ENGINEERING,EJECUTIVO,NaN,3315.1,5.6,95.0
8993,9998,23.0,MASCULINO,MEXICO,TERTIARY,ENGINEERING,PROGRAMACION,3.3,4437.8,0.9,87.0
8994,9999,36.0,MASCULINO,ARGENTINA,UNIVERSITARY,COMMERCIAL,DATA,3.4,4600.8,19.6,88.0


Columnas disponibles para el análisis

* Unnamed: 0
* EDAD
* GENERO
* RESIDENCIA
* NV_ESTUDIO
* ESTUDIO_PREV
* TRACK_DH
* AVG_DH
* MINUTES_DH
* EXPERIENCIA

* DIAS_EMP = El valor "target" que se va a predecir

In [ ]:
df_training_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8995 entries, 0 to 8994
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    8995 non-null   int64  
 1   EDAD          7668 non-null   float64
 2   GENERO        7620 non-null   object 
 3   RESIDENCIA    7638 non-null   object 
 4   NV_ESTUDIO    7623 non-null   object 
 5   ESTUDIO_PREV  7665 non-null   object 
 6   TRACK_DH      7714 non-null   object 
 7   AVG_DH        7651 non-null   float64
 8   MINUTES_DH    7619 non-null   float64
 9   EXPERIENCIA   7618 non-null   float64
 10  DIAS_EMP      8995 non-null   float64
dtypes: float64(5), int64(1), object(5)
memory usage: 773.1+ KB


# Paso 3. Algunos consejos antes de entrenar a un modelo:





1. Manejar valores perdidos
2. Manejar variables categóricas

## Remover columnas
**Aqui se decidió que solo se borrará la columna unnamed y AVG, ya que la residencia si pudiese influir en la regresión lineal**

In [ ]:
#df_training_dataset_1 = df_training_dataset.drop(columns=['Unnamed: 0', 'RESIDENCIA', 'NV_ESTUDIO', 'ESTUDIO_PREV', 'TRACK_DH'], inplace=False)
df_training_dataset_1 = df_training_dataset.drop(columns=['Unnamed: 0', 'AVG_DH'], inplace=False)
df_training_dataset_1.tail()

,EDAD,GENERO,RESIDENCIA,NV_ESTUDIO,ESTUDIO_PREV,TRACK_DH,MINUTES_DH,EXPERIENCIA,DIAS_EMP
8990,29.0,MASCULINO,ARGENTINA,TERTIARY,DEVELOPMENT,NaN,4701.6,9.1,86.0
8991,34.0,NaN,ARGENTINA,UNIVERSITARY,ENGINEERING,PROGRAMACION,4646.2,16.8,95.0
8992,28.0,FEMENINO,ARGENTINA,POST_GRADUATE,ENGINEERING,EJECUTIVO,3315.1,5.6,95.0
8993,23.0,MASCULINO,MEXICO,TERTIARY,ENGINEERING,PROGRAMACION,4437.8,0.9,87.0
8994,36.0,MASCULINO,ARGENTINA,UNIVERSITARY,COMMERCIAL,DATA,4600.8,19.6,88.0


Removendo valores NaN ---- se cambia por imputación de valor mas frecuente...

In [ ]:
#df_training_dataset_2 = df_training_dataset_1.dropna()
#df_training_dataset_2.tail()

from sklearn.impute import SimpleImputer
import numpy as np

impute_zeros = SimpleImputer(
    missing_values=np.nan,
    strategy='most_frequent',
    #strategy='constant',
    #fill_value=0,
    verbose=0,
    copy=True
)

print("Valores nulos antes de la transformación SimpleImputer: \n\n{}\n".format(df_training_dataset_1.isnull().sum(axis = 0)))

impute_zeros.fit(X=df_training_dataset_1)

# Reconstruyendo un Pandas DataFrame con los resultados
df_training_dataset_2 = pd.DataFrame.from_records(
    data=impute_zeros.transform(
        X=df_training_dataset_1
    ),
    columns=df_training_dataset_1.columns
)

print("Valores nulos antes de la transformación SimpleImputer: \n\n{}\n".format(df_training_dataset_2.isnull().sum(axis = 0)))

Valores nulos antes de la transformación SimpleImputer: 

EDAD            1327
GENERO          1375
RESIDENCIA      1357
NV_ESTUDIO      1372
ESTUDIO_PREV    1330
TRACK_DH        1281
MINUTES_DH      1376
EXPERIENCIA     1377
DIAS_EMP           0
dtype: int64

Valores nulos antes de la transformación SimpleImputer: 

EDAD            0
GENERO          0
RESIDENCIA      0
NV_ESTUDIO      0
ESTUDIO_PREV    0
TRACK_DH        0
MINUTES_DH      0
EXPERIENCIA     0
DIAS_EMP        0
dtype: int64



Trabajo con Variables Categóricas, Codificación de Variables Categóricas

**Se debe agregar la variable residencia, que tambien es categorica.**

In [ ]:
# Columnas One-hot-encoding del dataset usando el metodo de Pandas ``get_dummies`` (demontración)
#df_training_dataset_3 = pd.get_dummies(df_training_dataset_2, columns=['GENERO'])
df_training_dataset_3 = pd.get_dummies(df_training_dataset_2, columns=['GENERO','RESIDENCIA','NV_ESTUDIO','ESTUDIO_PREV','TRACK_DH'])
df_training_dataset_3.tail()

,EDAD,MINUTES_DH,EXPERIENCIA,DIAS_EMP,GENERO_FEMENINO,GENERO_MASCULINO,RESIDENCIA_ARGENTINA,RESIDENCIA_BRAZIL,RESIDENCIA_MEXICO,NV_ESTUDIO_POST_GRADUATE,NV_ESTUDIO_TERTIARY,NV_ESTUDIO_UNIVERSITARY,ESTUDIO_PREV_BUSINESS,ESTUDIO_PREV_COMMERCIAL,ESTUDIO_PREV_DEVELOPMENT,ESTUDIO_PREV_ENGINEERING,ESTUDIO_PREV_MARKETING,TRACK_DH_DATA,TRACK_DH_EJECUTIVO,TRACK_DH_MARKETING,TRACK_DH_PROGRAMACION
8990,29.0,4701.6,9.1,86.0,0,1,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0
8991,34.0,4646.2,16.8,95.0,0,1,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1
8992,28.0,3315.1,5.6,95.0,1,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0
8993,23.0,4437.8,0.9,87.0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1
8994,36.0,4600.8,19.6,88.0,0,1,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0


# Paso 4. Selección de columnas features y target (objetivo)

In [ ]:
df_training_dataset_3.columns

Index(['EDAD', 'MINUTES_DH', 'EXPERIENCIA', 'DIAS_EMP', 'GENERO_FEMENINO',
       'GENERO_MASCULINO', 'RESIDENCIA_ARGENTINA', 'RESIDENCIA_BRAZIL',
       'RESIDENCIA_MEXICO', 'NV_ESTUDIO_POST_GRADUATE', 'NV_ESTUDIO_TERTIARY',
       'NV_ESTUDIO_UNIVERSITARY', 'ESTUDIO_PREV_BUSINESS',
       'ESTUDIO_PREV_COMMERCIAL', 'ESTUDIO_PREV_DEVELOPMENT',
       'ESTUDIO_PREV_ENGINEERING', 'ESTUDIO_PREV_MARKETING', 'TRACK_DH_DATA',
       'TRACK_DH_EJECUTIVO', 'TRACK_DH_MARKETING', 'TRACK_DH_PROGRAMACION'],
      dtype='object')

In [ ]:
features = df_training_dataset_3[
    [
        #'EDAD', 'AVG_DH', 'MINUTES_DH', 'EXPERIENCIA',
        #'GENERO_FEMENINO', 'GENERO_MASCULINO'

       'EDAD', 'MINUTES_DH', 'EXPERIENCIA', 'DIAS_EMP',
       'GENERO_FEMENINO', 'GENERO_MASCULINO', 'RESIDENCIA_ARGENTINA',
       'RESIDENCIA_BRAZIL', 'RESIDENCIA_MEXICO', 'NV_ESTUDIO_POST_GRADUATE',
       'NV_ESTUDIO_TERTIARY', 'NV_ESTUDIO_UNIVERSITARY',
       'ESTUDIO_PREV_BUSINESS', 'ESTUDIO_PREV_COMMERCIAL',
       'ESTUDIO_PREV_DEVELOPMENT', 'ESTUDIO_PREV_ENGINEERING',
       'ESTUDIO_PREV_MARKETING', 'TRACK_DH_DATA', 'TRACK_DH_EJECUTIVO',
       'TRACK_DH_MARKETING', 'TRACK_DH_PROGRAMACION'
    ]
]

**Ahora, con la variable residencia agregada, nos percatamos que ahora hay mas parametros que considerar a la hora de realizar la regresión lineal.**

In [ ]:
target = df_training_dataset_3['DIAS_EMP']  ## NO CAMBIE EL NOMBRE DE LA VARIABLE "target".

In [ ]:
features

,EDAD,MINUTES_DH,EXPERIENCIA,DIAS_EMP,GENERO_FEMENINO,GENERO_MASCULINO,RESIDENCIA_ARGENTINA,RESIDENCIA_BRAZIL,RESIDENCIA_MEXICO,NV_ESTUDIO_POST_GRADUATE,NV_ESTUDIO_TERTIARY,NV_ESTUDIO_UNIVERSITARY,ESTUDIO_PREV_BUSINESS,ESTUDIO_PREV_COMMERCIAL,ESTUDIO_PREV_DEVELOPMENT,ESTUDIO_PREV_ENGINEERING,ESTUDIO_PREV_MARKETING,TRACK_DH_DATA,TRACK_DH_EJECUTIVO,TRACK_DH_MARKETING,TRACK_DH_PROGRAMACION
0,37.0,4131.5,21.3,97.0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1
1,40.0,4160.4,25.2,102.0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,1
2,35.0,4087.6,18.0,89.0,0,1,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0
3,33.0,4043.2,13.6,96.0,1,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0
4,29.0,4688.0,16.1,83.0,0,1,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8990,29.0,4701.6,9.1,86.0,0,1,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0
8991,34.0,4646.2,16.8,95.0,0,1,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1
8992,28.0,3315.1,5.6,95.0,1,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0
8993,23.0,4437.8,0.9,87.0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1


In [ ]:
target

0        97.0
1       102.0
2        89.0
3        96.0
4        83.0
        ...  
8990     86.0
8991     95.0
8992     95.0
8993     87.0
8994     88.0
Name: DIAS_EMP, Length: 8995, dtype: float64

# Paso 5. Separar Datos , dividir datos para una prueba ciega.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=133)

## Paso 6. Entreanamiento del modelo

Se entrena un modelo de regresión lineal simple

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
regression_model = LinearRegression().fit(X_train, y_train)

# Paso 7. Evaluación del modelo

In [ ]:
regression_model.score(X_test, y_test)

1.0

**Finalmente, el modelo original se habia decidio borrar la columna de residencia, sin embargo, ahora es considerada en este nuevo modelo, quedando un valor de regresión lineal de 1, valor que indica que los datos se ajustan perfectamente en una regla de forma lineal.**